Production (Importing, Functions, Team Spelling, Merging, Exporting)

In [37]:
# Import Libraries and Data
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 300)

# MDCM
mdcm = pd.read_csv('../data/mdcm/NCAA_Tourney_2002_2023.csv')
team_spellings = pd.read_csv('../data/mdcm/team_spellings.csv')

# March Machine Learning Mania 2024

# CBBData
selection_sunday_resume = pd.read_csv('../data/cbbdata/team/selection_sunday_resume.csv').query('year != 2020')

# Kaggle
coach_results = pd.read_csv('../data/kaggle/march_madness_data/coach_results.csv')
barttovik_home = pd.read_csv('../data/kaggle/march_madness_data/barttovik_home.csv')
barttovik_away = pd.read_csv('../data/kaggle/march_madness_data/barttovik_away.csv')
kenpom_barttovik = pd.read_csv('../data/kaggle/march_madness_data/kenpom_barttovik.csv')
shooting_splits = pd.read_csv('../data/kaggle/march_madness_data/shooting_splits.csv')
heat_check = pd.read_csv('../data/kaggle/march_madness_data/heat_check.csv')

In [38]:
# Define Functions
def merge_team_season(df: pd.DataFrame, df_merge_onto: pd.DataFrame, filter_df_merge_onto_year = None, title: str = None):
    """
    
        Function to merge teams and their seasons in college basketball with a bevy of alternative spellings, using team_spellings.csv.
    
        df (pd.DataFrame): The dataframe you'd like to establish as your left, or original df. Must contain 'teamname' and 'season' columns.
        
        df_merge_onto (pd.DataFrame): The dataframe you'd like to left merge onto df. Must contain 'team' and 'year' columns.
        
        filter_df_merge_onto_year (int): If there is a cutoff year for the df_merge_onto, the year of interest. This does not affect the 
                                        merge, just the reporting success messages.
    
    """
    # Print Title of Run For Terminal
    if title:
        print(title,'-----------------------\n')
        
    # If Oncoming Data Filtered By Specifc Year 
    if filter_df_merge_onto_year != None:
        df_post_cutoff = df[df['season'] >= filter_df_merge_onto_year]
        df_pre_cutoff = df[df['season'] < filter_df_merge_onto_year]
        df = df_post_cutoff
    
    # Find Null df Column Identifier For Oncoming DF
    for col_name in df_merge_onto.columns:
        if 'team' not in col_name.lower() and 'year' not in col_name.lower():
            col_null_match_identifier = col_name
            print('Column Null Match Identifier:', col_null_match_identifier,'\n')
            break
     
    # Set Both Teamname Columns To Lowercase Easier Merging
    for team_num in range(1,3):
        df[f'team{team_num}_teamname'] = df[f'team{team_num}_teamname'].str.lower()
    
    # Loop Through Team 1 and Team 2
    df_both_teams = pd.DataFrame()
    for team_num in range(1, 3):
        
        # Adjust Column Names Due To Team1 and Team2 (Remove 'team1_' or 'team1_'/'team2_')
        if team_num == 2:
            df_merge_onto.columns = df_merge_onto.columns.str[6:]
        df_merge_onto = df_merge_onto.add_prefix(f'team{team_num}_')
        
        # Establish df_merge_onto Team Column Name and Set to Lowercase
        df_merge_onto[f'team{team_num}_team'] = df_merge_onto[f'team{team_num}_team'].str.lower()
        
        # If Second Iteration, Find The Merge Columns and The Spellings Ones To Keep
        if team_num == 2:
            spellings = ['team2_teamname', 'season']
            for col in df.columns:
                if f"team{team_num}_name_spelling" in col: 
                    spellings.append(col)
            df = df[spellings].drop_duplicates()
            
        # Original Merge For Team 1 or 2
        print(f"Team {team_num} Merge ...\n")
        df_merged = pd.merge(df, df_merge_onto, how = 'left', left_on = [f'team{team_num}_teamname', 'season'], right_on = [f'team{team_num}_team', f'team{team_num}_year'])
        
        # Original Split Up Merged and Unmerged Data
        df_not_merged = df_merged[df_merged[f'team{team_num}_{col_null_match_identifier}'].isna() == True]
        df_merged = df_merged[df_merged[f'team{team_num}_{col_null_match_identifier}'].isna() == False]
        
        print(f'Original Team {team_num} Merge:', len(df), 'total rows.')
        print('Matched During Iteration:', len(df_merged)) 
        print('Unmatched Rows Remaining:', len(df_not_merged), '\n') 
        
        # Remove Columns That Didn't Merge Properly Based On Num of Columns
        # Reduce To Team and Year, Along with Alternate Spellings
        neg_col_count_df_merge_onto = df_merge_onto.shape[1] * -1
        df_not_merged = df_not_merged.iloc[:, :neg_col_count_df_merge_onto]
        
        # Loop Through Columnns To Fix The Merge
        print(f"Correcting Team {team_num} Merge ...\n")
        merge_complete, i = False, 1
        while merge_complete == False:
            
            # Perform Loop Everytime More Unmatched Columns Are Found
            team_season_loop = pd.merge(df_not_merged, df_merge_onto, how = 'left', left_on = [f'team{team_num}_name_spelling_{i}', 'season'], right_on = [f'team{team_num}_team', f'team{team_num}_year'])
            print(f'Team {team_num} Season Loop {i}:', len(team_season_loop), 'total rows.')
            
            # Split Up The Matched and Unmatched
            matched_df = team_season_loop[team_season_loop[f'team{team_num}_{col_null_match_identifier}'].isna() == False]
            print('Matched During Iteration:', len(matched_df)) 
            unmatched_df = team_season_loop[team_season_loop[f'team{team_num}_{col_null_match_identifier}'].isna() == True]
            print('Unmatched Rows Remaining:', len(unmatched_df), '\n') 
            
            # For The DataFrames With Data In The Column From Second DF, Add To team_season
            if len(matched_df) > 0:
                df_merged = pd.concat([df_merged, matched_df])
            # If There Are Still Null Rows, Throw Those Back In The Loop For The Next Iteration
            if len(unmatched_df) > 0:
                df_not_merged = unmatched_df.iloc[:, :neg_col_count_df_merge_onto]
            # If There Aren't Any Null Rows Left, End The Loop
            else:
                print(f'Success! Team {team_num} Merge Completed Early!\n')
                merge_complete = True
            if i == 11:
                merge_complete = True 
            i += 1
   
        # Concat Team 1 or 2 Onto Full DataFrame
        if team_num == 2:
            spellings = []
            for col in df_merged.columns:
                if 'team2_name_spelling' not in col:
                    spellings.append(col)
                
            df_both_teams = pd.merge(df_both_teams, df_merged[spellings], how = 'left', on = [f'team2_teamname', 'season'])
        else:
            df_both_teams = pd.concat([df_merged, df_not_merged])
    
    # Inspect Final Results
    print("Filter Views of Resulting DataFrame -------------------------\n")
    for team_num in range(1, 3):
        print(f'Team{team_num}:')
        if filter_df_merge_onto_year != None:
            # Gather Data About Merge Post Cutoff
            post_cutoff_rows = df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == False) & (df_both_teams['season'] >= filter_df_merge_onto_year)].shape[0]
            post_cutoff_rows_na = df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == True) & (df_both_teams['season'] >= filter_df_merge_onto_year)].shape[0]
            
            if post_cutoff_rows_na > 0:
                print(f'Oh No! There were {post_cutoff_rows} matches and {post_cutoff_rows_na} post {filter_df_merge_onto_year} null games.')
                
                # # Get Teamnames That Dont Merge
                # if pd.read_csv('../data/sample_spell.csv').shape[0] == pd.merge(pd.read_csv('../data/mdcm/team_spellings.csv'), 
                # pd.read_csv('../data/sample_spell.csv'), how = 'inner', left_on = 'name_spelling', right_on = 'team2_teamname').shape[0]:
                #         print("Despite all the mismatches, none of them were in the merged onto Dataframe.")
                # else:
                #     print("Here are the distinct team names from the 'df' dataframe that weren't merged properly:")
                #     print(df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == True) & (df_both_teams['season'] >= filter_df_merge_onto_year)][f'team{team_num}_teamname'].drop_duplicates().sort_values())
            else:
                print(f'Great! No Null Rows Post {filter_df_merge_onto_year}')
     
            # Gather Data About Merge Pre Cutoff
            pre_cutoff_rows = df_both_teams[(df_both_teams[f'team{team_num}_{col_null_match_identifier}'].isna() == False) & (df_both_teams['season'] < filter_df_merge_onto_year)].shape[0]
            if pre_cutoff_rows == 0:
                print(f'Great! No Matched Rows Pre {filter_df_merge_onto_year}\n')
                                
    # Concat Pre and Post Cutoff If Exists
    if filter_df_merge_onto_year != None:
        df_both_teams = pd.concat([df_both_teams, df_pre_cutoff], ignore_index=True)    
    
    return df_both_teams.drop(['team1_team','team1_year', 'team2_team', 'team2_year'], axis = 1)

# Ammend List To Remove Duplicate Columns And Retain One Set 
def drop_dup_columns(df: pd.DataFrame, dup_cols_keep: list[str]):
    same_data_columns = []
    for i in range(df.shape[1] - 1):
        for j in range(i + 1, df.shape[1]):
            col1, col2 = df.columns[i], df.columns[j]
            if df[col1].equals(df[col2]):
                same_data_columns.append((col1, col2))
    if not same_data_columns:
        print("No columns have the same data.")
    else:
        for col_pair in same_data_columns:
            if col_pair not in dup_cols_keep:
                df.drop(columns = col_pair[0], axis = 1, inplace = True)
                df.rename(columns = {col_pair[1]: col_pair[1].replace('_away', '')}, inplace = True)
    return df

In [39]:
# Create Round Column
mdcm['round'] = mdcm['slot'].str.extract(r'(\d+)')
mdcm['round'] = pd.to_numeric(mdcm['round'], errors='coerce')

# Adjust Team Spellings
team_spellings = team_spellings.pivot_table(index='team_id', columns=team_spellings.groupby('team_id').cumcount(), values='name_spelling', aggfunc='first')
team_spellings.columns = [f'name_spelling_{i + 1}' for i in range(team_spellings.shape[1])]
team_spellings.reset_index(inplace=True)

# Merge Team Spellings
team_spellings_t1 = team_spellings.add_prefix('team1_')
mdcm = pd.merge(mdcm, team_spellings_t1, how = 'inner', left_on = ['team1_id'], right_on = ['team1_team_id'])
team_spellings_t2 = team_spellings.add_prefix('team2_')
mdcm = pd.merge(mdcm, team_spellings_t2, how = 'inner', left_on = ['team2_id'], right_on = ['team2_team_id'])

In [40]:
# Merge MDCM and Selection Sunday (2008-2024)

print(mdcm.shape)
selection_sunday_resume = selection_sunday_resume[selection_sunday_resume['year'] != 'Year']
selection_sunday_resume['year'] = selection_sunday_resume['year'].astype(int)
selection_sunday_resume.drop(columns = ['net', 'seed'], axis = 1, inplace = True)
selection_sunday_resume.rename(columns={'score': 'team_score'}, inplace=True)
df = merge_team_season(mdcm, selection_sunday_resume, filter_df_merge_onto_year = 2008, title = 'MDCM and Selection Sunday (2008 - 2024)')
print(df.shape)

(1380, 129)
MDCM and Selection Sunday (2008 - 2024) -----------------------

Column Null Match Identifier: resume 

Team 1 Merge ...

Original Team 1 Merge: 996 total rows.
Matched During Iteration: 782
Unmatched Rows Remaining: 214 

Correcting Team 1 Merge ...

Team 1 Season Loop 1: 214 total rows.
Matched During Iteration: 0
Unmatched Rows Remaining: 214 

Team 1 Season Loop 2: 214 total rows.
Matched During Iteration: 9
Unmatched Rows Remaining: 205 

Team 1 Season Loop 3: 205 total rows.
Matched During Iteration: 101
Unmatched Rows Remaining: 104 

Team 1 Season Loop 4: 104 total rows.
Matched During Iteration: 37
Unmatched Rows Remaining: 67 

Team 1 Season Loop 5: 67 total rows.
Matched During Iteration: 6
Unmatched Rows Remaining: 61 

Team 1 Season Loop 6: 61 total rows.
Matched During Iteration: 2
Unmatched Rows Remaining: 59 

Team 1 Season Loop 7: 59 total rows.
Matched During Iteration: 0
Unmatched Rows Remaining: 59 

Team 1 Season Loop 8: 59 total rows.
Matched During It

C:\Users\Peter\AppData\Local\Temp\ipykernel_10500\1399182595.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'team{team_num}_teamname'] = df[f'team{team_num}_teamname'].str.lower()


In [43]:
df[df['team1_resume'].isna() == True]['team1_teamname'].drop_duplicates()

944              lehigh
945              mercer
946                 uab
947          st peter's
948         tx southern
949           albany ny
950       james madison
951             radford
952         f dickinson
953        cal poly slo
955           princeton
958        oral roberts
959           san diego
960      ark pine bluff
961             harvard
963            richmond
964         morehead st
965      ut san antonio
966          holy cross
967       fl gulf coast
968            marshall
969           oregon st
972         n dakota st
973            uc davis
974       unc asheville
977          w kentucky
979                ohio
981              furman
983                umbc
984           wright st
986      tam c. christi
988          norfolk st
989             hampton
990              nc a&t
992             vermont
993        mt st mary's
995       robert morris
996              Kansas
997         Michigan St
998             Indiana
999         Connecticut
1000            

In [ ]:

# Merge MDCM and Kenpom Barttovik (2008-2024)
kenpom_barttovik.columns = kenpom_barttovik.columns.str.lower()
kenpom_barttovik = kenpom_barttovik[['year', 'team', 'barthag', 'avg hgt', 'eff hgt', 'exp', 'talent', 'pppo', 'pppd', 'elite sos']]
df = merge_team_season(df, kenpom_barttovik, filter_df_merge_onto_year = 2008, title = 'MDCM and Kenpom Barttovik (2008 - 2024)')
print(df.shape)

# Merge Home Team Barttovik Data (2008-2024)
barttovik_home.columns = barttovik_home.columns.str.lower()
barttovik_home.drop(columns = ['seed', 'round', 'wab'], axis = 1, inplace = True)
barttovik_home.rename(columns={col: f'home_' + col for col in barttovik_home.columns[-1 * (barttovik_home.shape[1]):]}, inplace=True)
barttovik_home = barttovik_home[['home_year','home_team', 'home_badj em', 'home_badj o', 'home_badj d', 'home_barthag', 
                                 'home_badj t', 'home_pppo', 'home_pppd','home_3pt%', 'home_ft%']]
barttovik_home.rename(columns={f'home_team':'team', 'home_year':'year'}, inplace = True)
df = merge_team_season(df, barttovik_home, filter_df_merge_onto_year = 2008, title = 'MDCM and Home Barttovic Data (2008 - 2024)')
print(df.shape)

# Merge Away Team Barttovik Data (2008-2024)
barttovik_away.columns = barttovik_away.columns.str.lower()
barttovik_away.drop(columns = ['seed', 'round', 'wab'], axis = 1, inplace = True)
barttovik_away.rename(columns={col: f'away_' + col for col in barttovik_away.columns[-1 * (barttovik_away.shape[1]):]}, inplace=True)
barttovik_away = barttovik_away[['away_year','away_team', 'away_badj em', 'away_badj o', 'away_badj d', 'away_barthag', 
                                 'away_badj t', 'away_pppo', 'away_pppd','away_3pt%', 'away_ft%']]
barttovik_away.rename(columns={f'away_team':'team', 'away_year':'year'}, inplace = True)
df = merge_team_season(df, barttovik_away, filter_df_merge_onto_year = 2008, title = 'MDCM and Away Barttovic Data (2008 - 2024)')
print(df.shape)


In [ ]:

# Shooting Splits (2010-2024)
shooting_splits.columns = shooting_splits.columns.str.lower()
shooting_splits = shooting_splits.drop(['conf'], axis=1, errors='ignore')
df = merge_team_season(df, shooting_splits, filter_df_merge_onto_year = 2010, title = 'MDCM and Kenpom Barttovik (2013 - 2024)')

# Remove Rank From DataFrame
df = df.loc[:, ~df.columns.str.contains('rank', case=False)].dropna(axis=1, how='all')
df = df.loc[:, ~df.columns.str.contains('name_spelling')]
df.to_csv("../data/pipeline/merged_team_season.csv", index = False)
df.head()

In [6]:
df.shape

(1330, 151)

In [ ]:

# Merge MDCM and Kenpom Barttovik (2008-2024)
kenpom_barttovik.columns = kenpom_barttovik.columns.str.lower()
kenpom_barttovik = kenpom_barttovik[['year', 'team', 'barthag', 'avg hgt', 'eff hgt', 'exp', 'talent', 'pppo', 'pppd', 'elite sos']]
df = merge_team_season(df, kenpom_barttovik, filter_df_merge_onto_year = 2008, title = 'MDCM and Kenpom Barttovik (2008 - 2024)')

# Merge Home Team Barttovik Data (2008-2024)
barttovik_home.columns = barttovik_home.columns.str.lower()
barttovik_home.drop(columns = ['seed', 'round', 'wab'], axis = 1, inplace = True)
barttovik_home.rename(columns={col: f'home_' + col for col in barttovik_home.columns[-1 * (barttovik_home.shape[1]):]}, inplace=True)
barttovik_home = barttovik_home[['home_year','home_team', 'home_badj em', 'home_badj o', 'home_badj d', 'home_barthag', 
                                 'home_badj t', 'home_pppo', 'home_pppd','home_3pt%', 'home_ft%']]
barttovik_home.rename(columns={f'home_team':'team', 'home_year':'year'}, inplace = True)
df = merge_team_season(df, barttovik_home, filter_df_merge_onto_year = 2008, title = 'MDCM and Home Barttovic Data (2008 - 2024)')

# Merge Away Team Barttovik Data (2008-2024)
barttovik_away.columns = barttovik_away.columns.str.lower()
barttovik_away.drop(columns = ['seed', 'round', 'wab'], axis = 1, inplace = True)
barttovik_away.rename(columns={col: f'away_' + col for col in barttovik_away.columns[-1 * (barttovik_away.shape[1]):]}, inplace=True)
barttovik_away = barttovik_away[['away_year','away_team', 'away_badj em', 'away_badj o', 'away_badj d', 'away_barthag', 
                                 'away_badj t', 'away_pppo', 'away_pppd','away_3pt%', 'away_ft%']]
barttovik_away.rename(columns={f'away_team':'team', 'away_year':'year'}, inplace = True)
df = merge_team_season(df, barttovik_away, filter_df_merge_onto_year = 2008, title = 'MDCM and Away Barttovic Data (2008 - 2024)')

# Shooting Splits (2010-2024)
shooting_splits.columns = shooting_splits.columns.str.lower()
shooting_splits = shooting_splits.drop(['conf'], axis=1, errors='ignore')
df = merge_team_season(df, shooting_splits, filter_df_merge_onto_year = 2010, title = 'MDCM and Kenpom Barttovik (2013 - 2024)')

# Remove Rank From DataFrame
df = df.loc[:, ~df.columns.str.contains('rank', case=False)].dropna(axis=1, how='all')
df = df.loc[:, ~df.columns.str.contains('name_spelling')]
df.to_csv("../data/pipeline/merged_team_season.csv", index = False)
df.head()

Testing

In [256]:
# Upset Frequencies (2023-2008)
upset_counts = pd.read_csv('../data/kaggle/march_madness_data/upset_counts.csv')
upset_counts.columns = upset_counts.columns.str.lower()
upset_counts.head()

for row in upset_counts.shape[0]:
    upset_counts_no_current_year = upset_counts[upset_counts['year'] != row]

,year,first round,second round,sweet 16,elite 8,final 4,total
0,2023,5,4,3,2,0,14
1,2022,7,5,4,0,1,17
2,2021,9,6,2,1,0,18
3,2019,8,0,1,2,0,11
4,2018,6,5,3,1,0,15


In [76]:
# Pre and During NCAA Tourney Split of Data, Include Rolling Averages For Team Performance (Values as of Date)

# Import Team Ratings By Day (2015-2019)
team_ratings = pd.read_csv('../data/cbbdata/team/team_ratings.csv').query('year != 2020')
team_ratings['date']= pd.to_datetime(team_ratings['date'])

# Split Daily Team Ranknings By Before/After Selection Sunday
ss_dict = {2024: "2024-3-17", 2023: "2023-3-12", 2022: "2022-3-13", 
           2021: "2021-3-14", 2019: "2019-3-17", 2018: "2018-3-11", 
           2017: "2017-3-12", 2016: "2016-3-13", 2015: "2015-3-15"}

# Create A Pre and During NCAA Tournament Day By Day Ratings
team_rating_pre_ncaa = pd.DataFrame(columns = team_ratings.columns)
team_rating_ncaa = pd.DataFrame(columns = team_ratings.columns)
for year, ss_date in ss_dict.items():
    team_rating_pre_ncaa = pd.concat([team_rating_pre_ncaa, team_ratings[(team_ratings['year'] == year) & (team_ratings['date'] < ss_date)]])    
    team_rating_ncaa = pd.concat([team_rating_ncaa, team_ratings[(team_ratings['year'] == year) & (team_ratings['date'] > ss_date)]])
    
# Look At Team Rating By Day and Calculate Rolling Adj Offensive Rank
team_rating_pre_ncaa['rolling_avg_adj_o_rk'] = team_rating_pre_ncaa.groupby(['team', 'year'])['adj_o'].transform(lambda x: x.rolling(window=3).mean())
team_rating_pre_ncaa[(team_rating_pre_ncaa['team'] == 'Kentucky') & (team_rating_pre_ncaa['year'] == 2015)].head()

In [218]:
game_factors = pd.read_csv('../data/cbbdata/game/game_factors.csv').query('year != 2020')

game_factors['date']= pd.to_datetime(game_factors['date'])
team_rating_pre_ncaa['date']= pd.to_datetime(team_rating_pre_ncaa['date'])

test = pd.merge(game_factors[game_factors['year'] > 2015], team_rating_pre_ncaa.drop(columns = ['conf', 'year']), how = 'inner', on = ['team', 'date'])
test = test.rename(columns = {'adj_o_x':'adjo_game', 'adj_d_x':'adjd_game','adj_o_y':'adjo_season_date', 'adj_d_y':'adjd_season_date'})
test.head()

C:\Users\Peter\AppData\Local\Temp\ipykernel_24000\3081602681.py:1: DtypeWarning: Columns (25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  game_factors = pd.read_csv('../data/cbbdata/game/game_factors.csv').query('year != 2020')


,date,type,team,conf,opp,opp_conf,loc,result,pts_scored,pts_allowed,adjo_game,adjd_game,off_ppp,off_efg,off_to,off_or,off_ftr,def_ppp,def_efg,def_to,def_or,def_ftr,game_score,season,tempo,game_id,coach,opp_coach,year,location,avg_marg,rank,record,barthag,adjo_season_date,adj_o_rk,adjd_season_date,adj_d_rk,adj_tempo,adj_tempo_rk,proj_record,proj_conf_record,wab,wab_rk,rec,proj_rec,proj_conf_rec,cur_rk,change,rolling_avg_adj_o_rk
0,2016-03-12,conf_t,Akron,MAC,Buffalo,MAC,N,L,61,64,91.4,96.7,91.9,50.0,18.1,16.7,3.4,96.4,45.9,22.6,34.2,13.1,34.2,2016.0,66.4,BuffaloAkron3-12,Keith Dambrot,Nate Oats,2016,NaN,NaN,92,25-8,0.720127,109.546395,81,100.903669,126,68.500901,202,25.3-8.7,13-5,-1.092046,79,NaN,NaN,NaN,0.0,0.0,109.961430
1,2016-03-12,conf_t,Buffalo,MAC,Akron,MAC,N,W,64,61,101.5,86.0,96.4,45.9,22.6,34.2,13.1,91.9,50.0,18.1,16.7,3.4,87.1,2016.0,66.4,BuffaloAkron3-12,Nate Oats,Keith Dambrot,2016,NaN,NaN,139,19-14,0.567722,104.973415,153,102.514686,151,72.986535,28,19.1-14.9,10-8,-5.527029,130,NaN,NaN,NaN,0.0,0.0,104.926304
2,2016-03-12,conf_t,Cal St. Bakersfield,WAC,New Mexico St.,WAC,N,W,57,54,106.8,90.5,96.1,47.2,18.5,33.3,24.1,91.0,43.8,23.6,35.1,45.8,87.0,2016.0,59.3,Cal St. BakersfieldNew Mexico St.3-12,Rod Barnes,Marvin Menzies,2016,NaN,NaN,94,21-8,0.714282,102.855153,181,94.978036,29,68.587124,196,21.2-8.8,11-3,-3.333534,98,NaN,NaN,NaN,0.0,0.0,102.721049
3,2016-03-12,conf_t,New Mexico St.,WAC,Cal St. Bakersfield,WAC,N,L,54,57,95.8,98.9,91.0,43.8,23.6,35.1,45.8,96.1,47.2,18.5,33.3,24.1,40.9,2016.0,59.3,Cal St. BakersfieldNew Mexico St.3-12,Marvin Menzies,Rod Barnes,2016,NaN,NaN,112,21-10,0.651555,101.791294,203,96.399421,46,66.535250,299,21.1-10.9,13-1,-3.889725,108,NaN,NaN,NaN,0.0,0.0,101.668366
4,2016-03-12,conf_t,Connecticut,Amer,Temple,Amer,N,W,77,62,122.3,93.9,117.0,58.0,13.7,26.9,26.8,94.2,39.1,12.2,35.6,34.4,95.4,2016.0,65.8,ConnecticutTemple3-12,Kevin Ollie,Fran Dunphy,2016,NaN,NaN,33,23-10,0.875603,110.127964,70,92.939894,14,67.075610,274,24.5-11.5,11-7,1.166640,39,NaN,NaN,NaN,0.0,0.0,109.501952
